<a href="https://colab.research.google.com/github/imthelizardking/CMP784--Final--Project/blob/main/suggestion_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import and install libraries and dependencies

In [1]:
import tensorflow as tf
import numpy as np
import os
import json
import collections
import pickle
from tqdm import tqdm
import pylab
use_colab = True # belki silinecek
TINY = 1e-30
EPS = 1e-4
nax = np.newaxis

Connect to gdrive
(Dataseti gdrive'dan çekiyordum, datalar local makineden çekileceği için bu kısım iptal edilebilir)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Import data (Yine bu kısma da adres olarak dataset'in local makinedeki adresi verilmeli)

In [3]:
# import data, from gdrive
dirJson = "/content/gdrive/MyDrive/cmp784_project_literature/colab repo/test_inputs/"
%cd "/content/gdrive/MyDrive/cmp784_project_literature/colab repo/test_inputs"

/content/gdrive/MyDrive/cmp784_project_literature/colab repo/test_inputs


Burada, suggestion model trainer'a uygun matrisler üretiliyor.

In [12]:
# labels_vector_ids: Bir image/annotation pair'ındaki itemler vektörün içerisine id'leriyle yazılıyor,
# eğer pair'daki item'lar "shorts": (id=6) ve "long sleeve top" (id=1) ise, matrisin ilgili satırı [6, 1, 0, 0, ..., 0] oluyor
# labels_vector_ids_size: her bir pair'da kaç tane item olduğunun bilgisi, yukarıdaki örnek için bu değer 2,
json_files = [pos_json for pos_json in os.listdir(dirJson) if pos_json.endswith('.json')] # get all .json anno. files from dir
NUM_FEATURES = 13 # number of features (types, color, style etc.)
labels_vector_ids = np.zeros((len(json_files),NUM_FEATURES))
labels_vector_ids_size = np.zeros(len(json_files))
counter_outer = 0 # anno. counter
typeDict = { # dictionary'imiz, deepfashion2'nun annotationlarındaki id'lere göre
    "short sleeve top": 0,
    "long sleeve top": 1,
    "short sleeve outwear": 2,
    "long sleeve outwear": 3,
    "vest": 4,
    "sling": 5,
    "shorts": 6,
    "trousers": 7,
    "skirt": 8,
    "short sleeve dress": 9,
    "long sleeve dress": 10,
    "vest dress": 11,
    "sling dress": 12,
}
for json_ in json_files:
  jsonFile = open(json_, 'r')
  values = json.load(jsonFile)
  jsonFile.close()
  counter_inner = 0
  for (k,v) in values.items():
    if k.startswith('item'):    
      labels_vector_ids[counter_outer][counter_inner] = typeDict[v["category_name"]]
      counter_inner = counter_inner + 1
  labels_vector_ids_size[counter_outer] = counter_inner
  counter_outer = counter_outer + 1
print("labels_vector_ids_size: ", labels_vector_ids_size)
#labels_vector_ids = labels_vector_ids + 1 # hepsine +1 yapmak gerekebilir, 1 olsun min
print("labels_vector_ids: ", labels_vector_ids)

labels_vector_ids_size:  [2. 1. 2. 2. 2. 1. 1. 1. 1. 2. 2. 1. 1. 2. 2. 1. 2. 2. 1. 2. 1. 1. 1. 1.
 1. 2. 1. 2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 1. 1. 2. 1. 2. 1. 2. 2.
 2. 1. 2. 3. 1. 2. 2. 1. 1. 2. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1.
 2. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 2. 2. 2. 1. 1. 1. 1. 1.
 1. 1. 2. 1. 2. 1. 2. 2. 2. 1. 1. 2. 1. 2. 2. 1. 2. 2. 2. 1. 1. 1. 1. 2.
 1. 2. 2. 1. 2. 1. 1. 1. 1. 2. 1. 1. 2. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 2. 1. 2. 1. 1. 2. 1. 2. 1. 1. 2. 2. 1. 2.
 2. 2. 2. 2. 1. 1. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 2. 2. 1. 1. 2. 1. 2. 2.
 1. 2. 1. 1. 1. 1. 2. 1.]
labels_vector_ids:  [[8. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [6. 0. 0. ... 0. 0. 0.]
 ...
 [3. 0. 0. ... 0. 0. 0.]
 [7. 3. 0. ... 0. 0. 0.]
 [9. 0. 0. ... 0. 0. 0.]]


Compatility matrix (modified Co-occurrene matrix):

In [69]:
# Burada, co-occurence matrisini hesaplayıp, daha sonra bu matrise heart ve comment rating'lerini ekleyerek logaritmasını alıyoruz,
# çıktı log_compatibility_matrix oluyor
def calculate_log_co_occurence(labels_vector_ids, labels_vector_ids_size): # rating comment ve star eklenecek
  dataset_size = len(labels_vector_ids)
  scaler = 3
  rating_heart = scaler * np.random.random_sample((13, 13, 13)) # heart rating ve comment rating elimde olmadığı için bunları random generate ettim,                                                                                            
  rating_comment = scaler * np.random.random_sample((13, 13, 13)) # bunlar elimize geçince bu method'a input olarak verebiliriz.
  log_co_occurence = np.zeros((13, 13, 13)) 
  for counter_temp in range(len(labels_vector_ids_size)):
    i,j,k = 0,0,0
    i,j,k = labels_vector_ids[counter_temp][0], labels_vector_ids[counter_temp][1], labels_vector_ids[counter_temp][2]
    log_co_occurence[int(i)][int(j)][int(k)] +=1
    # add ratings assignment with scaler/normalizer
  
  # ratingleri + olarak mı * olarak da düşünebiliriz, benim gdrive'dan çekebildiğim dataset çok küçük olduğu için + dedim şimdilik, ama *'ya çekebiliriz büyük datasette
  #log_co_occurence = np.multiply(np.multiply(log_co_occurence, rating_heart), rating_comment)
  log_co_occurence = log_co_occurence + rating_heart + rating_comment
  delta_smoothing = 0.5  # A hyperparameter.  You can play with this if you want.
  log_co_occurence += delta_smoothing  # Add delta so log doesn't break on 0's.
  # maybe normalize
  log_compatibility_matrix = np.log(log_co_occurence)
  return log_compatibility_matrix

Split data into two and create compatibility matrix for train and valid data

In [70]:
[data_train, data_valid] =  np.array_split(labels_vector_ids,2)
[data_train_size, data_valid_size] = np.array_split(labels_vector_ids_size,2)
log_co_occurence_train = calculate_log_co_occurence(data_train,data_train_size)
print(log_co_occurence_train.shape)
print("break")
log_co_occurence_valid = calculate_log_co_occurence(data_valid, data_valid_size)
#print(log_co_occurence_valid)

(13, 13, 13)
break


New appr.

Train helpers:

In [87]:
def loss_GLoVE(W, b, log_co_occurence):
  "Compute the GLoVE loss."
  n,_,_ = log_co_occurence.shape
  _,_,emb_dim = W.shape
  W_t = np.transpose(W,(0,emb_dim-1,1))
  b_t = np.transpose(b,(0,emb_dim-1,1))
  return np.sum((W @ W_t + b @ np.ones([1,n]) + np.ones([n,1])@b_t - log_co_occurence)**2)
  
def grad_GLoVE(W,  b, log_co_occurence):
  "Return the gradient of GLoVE objective w.r.t W and b."
  "INPUT: W - Vxd; b - Vx1; log_co_occurence: VxV"
  "OUTPUT: grad_W - Vxd; grad_b - Vx1"
  # INPUT: W- 13x13xd; b - 13x13x1 log_comp: 13X13X13
  # OUTPUT: grad_W - 13x13x1 grad_b: 13x13x1
  n,_,_ = log_co_occurence.shape # n = #of item types, 13
  _,_,emb_dim = W.shape
  ###########################   YOUR CODE HERE  ##############################
  print(W.shape)
  W_t = np.transpose(W,(0,emb_dim-1,1))
  b_t = np.transpose(b,(0,emb_dim-1,1))
  log_t = np.transpose(log_co_occurence,(0,emb_dim-1,1))
  
  grad_W = 2*(W @ W_t + b @ np.ones([1,n]) + np.ones([n,1])@b_t - log_co_occurence) @ W
  print(log_co_occurence.shape)
  #grad_b = 2*(np.ones([1,100]) @ (W @ W_t + b @ np.ones([1,n]) + np.ones([n,1])@b_t - log_t))
  #grad_b = np.transpose(grad_b,emb_dim,1)
  grad_b = np.zeros((13,13,1))
  ############################################################################
  return grad_W, grad_b

def train_GLoVE(W, b, log_co_occurence_train, log_co_occurence_valid, n_epochs, do_print=False):
  #W:13x13xd b:13x13x1
  n,_,_ = log_co_occurence_train.shape
  learning_rate = 0.2 / n  # A hyperparameter.  You can play with this if you want.
  for epoch in range(n_epochs):
    grad_W, grad_b = grad_GLoVE(W, b, log_co_occurence_train)
    W -= learning_rate * grad_W
    b -= learning_rate * grad_b
    train_loss, valid_loss = loss_GLoVE(W, b, log_co_occurence_train), loss_GLoVE(W, b, log_co_occurence_valid)
    if do_print:
      print(f"Train Loss: {train_loss}, valid loss: {valid_loss}, grad_norm: {np.sum(grad_W**2)}")
  return W, b, train_loss, valid_loss

Trainer:

Set Trainer Parameters and Train

In [89]:
np.random.seed(1)
n_epochs = 10  # 500 A hyperparameter.  You can play with this if you want.
embedding_dims = np.array([3])  # Play with this
final_train_losses, final_val_losses = [], []  # Store the final losses for graphing
W_final_2d, b_final_2d = None, None
do_print = True  # If you want to see diagnostic information during training
for embedding_dim in tqdm(embedding_dims):
  init_variance = 0.5  # A hyperparameter.  You can play with this if you want.
  W = init_variance * np.random.normal(size=(13, 13, embedding_dim))
  b = init_variance * np.random.normal(size=(13, 13, 1))
  if do_print:
    print(f"Training for embedding dimension: {embedding_dim}")
  W_final, b_final, train_loss, valid_loss = train_GLoVE(W, b, log_co_occurence_train, log_co_occurence_valid, n_epochs, do_print=False)
  if embedding_dim == 2:
    # Save a parameter copy if we are training 2d embedding for visualization later
    W_final_2d = W_final
    b_final_2d = b_final
  final_train_losses += [train_loss]
  final_val_losses += [valid_loss]
  if do_print:
    print(f"Final validation loss: {valid_loss}")

100%|██████████| 1/1 [00:00<00:00, 243.84it/s]

Training for embedding dimension: 3
(13, 13, 3)
(13, 13, 13)
(13, 13, 3)
(13, 13, 13)
(13, 13, 3)
(13, 13, 13)
(13, 13, 3)
(13, 13, 13)
(13, 13, 3)
(13, 13, 13)
(13, 13, 3)
(13, 13, 13)
(13, 13, 3)
(13, 13, 13)
(13, 13, 3)
(13, 13, 13)
(13, 13, 3)
(13, 13, 13)
(13, 13, 3)
(13, 13, 13)
Final validation loss: 565.6582891768307


Plot training metrics:

In [ ]:
pylab.loglog(embedding_dims, final_train_losses)
pylab.xlabel("Embedding Dimension")
pylab.ylabel("Training Loss")
pylab.legend()

In [ ]:
pylab.loglog(embedding_dims, final_val_losses)
pylab.xlabel("Embedding Dimension")
pylab.ylabel("Validation Loss")
pylab.legend()

**PREDICTOR**

Process input

In [ ]:
test_input = np.array([1,0,0,0,0,0,0,0,0,0,0,0,0]) + 1
test_input_processed = np.zeros(len(test_input))
counter_distinct = 0
for e in range(len(test_input)):
  if e!=0:
    test_input_processed[counter_distinct] = e
    counter_distinct = counter_distinct + 1

Create Combinations of Input

In [1]:
def createCombinations(test_input_processed, counter_distinct):
  counter_array = 0
  if counter_distinct==0 or counter_distinct==1:
    print("Can't make any prediction for item size smaller than 2")
    return
  else:
    if counter_distinct==2:
      test_array = np.zeros(2)
      test_array[0] = test_array[0] + test_input_processed[0]
      test_array[1] = test_array[1] + test_input_processed[1]
    elif counter_distinct==3:
      test_array = np.zeros(3)
      test_array[0] = test_array[0] + test_input_processed[0]
      test_array[1] = test_array[1] + test_input_processed[1]
      test_array[2] = test_array[2] + test_input_processed[2]              

Predictor:

In [ ]:
def predictor():
  

Run Preditor for each combination

Choose b/w most probable results

Feed suggestion into language model

Output image, bb and textual